# Random Forest

In [1]:
import os
from datetime import datetime
import tensorflow as tf
import numpy as np
import json
import pickle
from sklearn.model_selection import train_test_split

In [0]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
base_path = './gdrive/My Drive/Colab Notebooks'

In [12]:
data_in_path = base_path + '/data_in/'
data_out_path = base_path + '/data_out/'
clean_train_review = 'clean_train_data.pkl'
train_label_data = 'nsmc_train_label.npy'
data_configs = 'data_configs.json'

clean_train_review = pickle.load(open(data_in_path+ clean_train_data, 'rb'))
label_data = np.load(open(data_in_path + train_label_data, 'rb'))
prepro_configs = json.load(open(data_in_path + data_configs, 'r'))

In [14]:
import gensim

In [16]:
model = gensim.models.Word2Vec(clean_train_review, min_count=10, size=100, window=5)

In [20]:
print(model.wv.vocab.keys())

dict_keys(['더빙', '진짜', '짜증나다', '목소리', '흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다', '너', '무재', '다그', '래서', '보다', '추천', '다', '교도소', '이야기', '구먼', '솔직하다', '재미', '없다', '평점', '조정', '사이', '그', '돋보이다', '스파이더맨', '에서', '늙다', '보이다', '하다', '너무나도', '이쁘다', '막', '걸음', '마', '떼다', '세', '부터', '초등학교', '학년', '살다', 'ㅋㅋㅋ', '별', '반개', '도', '아깝다', '움', '원작', '긴장감', '을', '제대로', '살리다', '욕', '나오다', '연', '기', '생활', '몇', '년', '인지', '정말', '발', '로', '해도', '그것', '보단', '낫다', '납치', '감금', '만', '반복', '드라마', '가족', '못', '사람', '모', '엿', '네', '액션', '있다', '안되다', '왜케', '낮다', '꽤', '볼', '데', '헐리우드', '식', '화려하다', '너무', '길들이다', '걍', '인피니트', '짱', '이다', '볼때', '마다', '눈물나다', '죽다', '년대', '향수', '자', '극', '감성', '절제', '멜로', '달인', '울면', '서', '손', '고', '때', '뛰다', '치다', '올', '뻔', '이범수', '드럽다', '담백하다', '깔끔하다', '좋다', '신', '문', '사', '로만', '자꾸', '잊어버리다', '취향', '존중', '다지', '내생', '에', '극장', '중', '가장', '노잼', '노', '감동', '임', '스토리', '어거지', 'ㄱ', '냥', '매번', '긴장', '되다', '재밋음', 'ㅠㅠ', '차다', '웃기다', '바스코', '이기', '면', '락스', '코', '라고', '끄

In [18]:
model.most_similar('더빙')

/Users/chojunghyun/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('성우', 0.8012272119522095),
 ('연예인', 0.7288280725479126),
 ('개그맨', 0.6865183115005493),
 ('발음', 0.6077364683151245),
 ('전문', 0.5948984026908875),
 ('아이돌', 0.5948599576950073),
 ('한국어', 0.5820659399032593),
 ('너무하다', 0.5736197233200073),
 ('캐스팅', 0.5587539672851562),
 ('써다', 0.557826042175293)]

In [19]:
model.most_similar('초딩')

/Users/chojunghyun/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('초등학생', 0.8344471454620361),
 ('중딩', 0.79826420545578),
 ('초등학교', 0.7509734630584717),
 ('중학생', 0.7430744171142578),
 ('중학교', 0.6754841804504395),
 ('학년', 0.6751827001571655),
 ('고딩', 0.6495320796966553),
 ('발톱', 0.6380213499069214),
 ('유치원', 0.6372218132019043),
 ('추석', 0.6336464285850525)]

In [21]:
model.most_similar('연기')

/Users/chojunghyun/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('연기력', 0.8375692367553711),
 ('캐스팅', 0.7339506149291992),
 ('발연기', 0.6976484060287476),
 ('역할', 0.6602493524551392),
 ('열연', 0.6403608322143555),
 ('목소리', 0.6082243919372559),
 ('명연기', 0.6064327955245972),
 ('케미', 0.588096022605896),
 ('조연', 0.5770258903503418),
 ('소화', 0.5656832456588745)]

In [27]:
vocab = set(model.wv.vocab.keys())

In [29]:
from tqdm import tqdm_notebook

In [73]:
len(clean_train_review[1])

11

In [95]:
sentence_vector = []

for sent in tqdm_notebook(clean_train_review):
    vector = np.mean([model.wv.word_vec(word) for word in sent if word in vocab], 0)
    if vector.shape == (100,):
        sentence_vector.append(vector)
    else:
        sentence_vector.append(np.zeros((100,)))

/Users/chojunghyun/anaconda3/envs/tensorflow/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/chojunghyun/anaconda3/envs/tensorflow/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [97]:
sentence_vector = np.array(sentence_vector)

In [98]:
print(sentence_vector.shape)

(150000, 100)


In [99]:
TEST_SPLIT = 0.1
RNG_SEED = 100
VOCAB_SIZE = prepro_configs['vocab_size']
EMB_SIZE = 128
BATCH_SIZE = 50 # 16 -> 50
NUM_EPOCHS = 1

input_train, input_eval, label_train, label_eval = train_test_split(sentence_vector,
                                                                    label_data,
                                                                    test_size=TEST_SPLIT,
                                                                    random_state=RNG_SEED)

In [102]:
print(input_train.shape)
print(label_train.shape)

(135000, 100)
(135000,)


In [103]:
from sklearn.ensemble import RandomForestClassifier


# 랜덤 포레스트 분류기에  100개 의사 결정 트리를 사용한다.
forest = RandomForestClassifier(n_estimators = 100) 

# 단어 묶음을 벡터화한 데이터와 정답 데이터를 가지고 학습을 시작한다.
forest.fit(input_train, label_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [104]:
print(f"Accuracy: {forest.score(input_eval, label_eval)}")  # 검증함수로 정확도 측정

Accuracy: 0.8087333333333333
